<font size = "5"> **EELS_Tools:  [Spectroscopy](Ch4-Spectroscopy.ipynb)** </font>

<hr style="height:1px;border-top:4px solid #FF8200" />

# Analysis of Core-Loss EELS Spectra
<br>


[Download](https://raw.githubusercontent.com/pycroscopy/pyTEMlib/main/notebooks/EELS/Analysis_Core_Loss.ipynb)
 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
    https://colab.research.google.com/github/pycroscopy/pyTEMlib/blob/main/notebooks/EELS/Analysis_Core_Loss.ipynb)
    
part of 

<font size = "5"> **[pyTEMlib](https://pycroscopy.github.io/pyTEMlib/about.html)**</font>

a [pycroscopy](https://pycroscopy.github.io/pycroscopy/about.html) ecosystem package


Notebook by Gerd Duscher, 2023

Microscopy Facilities<br>
Institute of Advanced Materials & Manufacturing<br>
The University of Tennessee, Knoxville

Model based analysis and quantification of data acquired with transmission electron microscopes




## Content

Quantitative determination of chemical composition from a core-loss EELS spectrum


Relavant notebooks in course [MSE 672: Introduction to TEM]()  give an in depth explanation of the method.

Please cite:

[M. Tian et  al. *Measuring the areal density of nanomaterials by electron energy-loss spectroscopy*
Ultramicroscopy Volume 196, 2019, pages 154-160](https://doi.org/10.1016/j.ultramic.2018.10.009)

as a reference of this quantification method.

## Prerequesites
This notebook will only run with  ``pyTEMlib version 0.2023.5.0`` or higher.

### Install missing packages
This can take a while for the first time. This code will have to run at least once on your computer and for each colab session.

In [3]:
import sys

from pkg_resources import get_distribution, DistributionNotFound

def test_package(package_name):
    """Test if package exists and returns version or -1"""
    try:
        version = (get_distribution(package_name).version)
    except (DistributionNotFound, ImportError) as err:
        version = '-1'
    return version


# pyTEMlib setup ------------------
if test_package('pyTEMlib') < '0.2023.5.0':
    print('installing pyTEMlib')
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/SciFiReaders.git@main -q
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/pyTEMlib.git@main -q
    
# ------------------------------
print('done')

installing pyTEMlib
done


### Load the relevant packages

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pylab as plt 
import sys

if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
else:
    %gui qt5
sys.path.insert(0, '../../')
%load_ext autoreload
%autoreload 2
import pyTEMlib
import pyTEMlib.file_tools as ft     # File input/ output library
import pyTEMlib.image_tools as it
import pyTEMlib.eels_tools  as eels        # EELS methods 

import pyTEMlib.interactive_eels as ieels  # Dialogs for EELS input and quantification

# For archiving reasons it is a good idea to print the version numbers out at this point
print('pyTEMlib version: ', pyTEMlib.__version__)
if pyTEMlib.__version__ < '0.2023.3.0':
    print('This version of pyTEMlib will not work with this notebook')
    print('Please install new pyTEMlib version with code cell above first and then restart this notebook!')
    
__notebook__ = 'analyze_core_loss'
__notebook_version__ = '2023_03_20'


if 'google.colab' in sys.modules:
    drive.mount("/content/drive")

You don't have igor installed.     If you wish to open igor files, you will need to install it     (pip install igor) before attempting.
You don't have gwyfile installed.     If you wish to open .gwy files, you will need to      install it (pip install gwyfile) before attempting.
Symmetry functions of spglib enabled
SimpleITK not installed; Registration Functions for Image Stacks not available
install with: conda install -c simpleitk simpleitk 
pyTEMlib version:  0.2023.5.1


## Load and plot a spectrum

As an example we load the spectrum **1EELS Acquire (high-loss).dm3** from the *example data* folder.

Please see [Loading an EELS Spectrum](LoadEELS.ipynb) for details on storage and plotting.

First a dialog to select a file will apear.

Then the spectrum plot and ``Spectrum Info`` dialog will appear, in which we set the experimental parameters.

Please use the ``Set Energy Scale`` button to change the energy scale. When pressed a new dialog and a cursor will appear in which one is able to set the energy scale based on known features in the spectrum.


In [106]:
# -----Input -------#
load_example = True

if load_example:
    datasets = ft.open_file('../../example_data/EELS_STO3.hf5')
else:  
    fileWidget = ft.FileWidget()

edges
0
all_edges
L1
L2
L3
1
all_edges
K1
fit_area
model
experiment
peak_fit
peaks
0
1
2
3
4
5
white_line_ratios
white_line_sums
white_lines
DM
DocumentObjectList
0
AnnotationGroupList
ImageDisplayInfo
CalibrationSliceId
DimensionLabels
GroupList
0
GroupToDisplay
MainSliceId
SliceList
0
ImageToGroup
SliceId
ObjectTags
DocumentTags
Image Behavior
UnscaledTransform
ZoomAndMoveTransform
ImageData
Calibrations
Brightness
Dimension
0
Dimensions
ImageSourceList
0
Extra Slice Info
0
Id
Id
ImageTags
Acquisition
Device
CCD
Configuration
Transpose
Frame
Area
Transform
Transform List
0
Transpose
CCD
Intensity
Transform
Transform List
0
1
Parameters
Base Detector
Detector
Environment
High Level
Shutter
Transform
Objects
0
DataBar
Custom elements
EELS
Acquisition
Experimental Conditions
Meta Data
Microscope Info
Items
0
1
2
MinVersionList
0
Page Behavior
PageTransform
SentinelList
Thumbnails
0
UniqueID
experiment
DM
DocumentObjectList
0
AnnotationGroupList
ImageDisplayInfo
CalibrationSliceId
Dimens

In [107]:
if not load_example:
    datasets = fileWidget.datasets
infoWidget= ieels.InfoWidget(datasets)


AppLayout(children=(GridspecLayout(children=(Dropdown(description='Main Dataset:', layout=Layout(grid_area='wi…

## Chemical Composition 
The fit of the cross-section and background to the spectrum results in the chemical composition. If the calibration is correct this composition is given as areal density in atoms/nm$^2$




In [59]:
print('Auto Quantification')
eels.auto_chemical_composition(infoWidget.dataset)

Auto Quantification

Relative composition: 
Ti: 18.3%  O: 81.7%  


###  Fit of Data
A dialog window will open, enter the elements first (0 will open a periodic table) and press 
``Fit Composition`` button (bottom right). Adjust parameters as needed and check fit by pressing the ``Fit Composition`` button again.

Select the ``Region`` checkbox to see which parts of the spectrum you choose to fit.

Changing the multiplier value will make a simulation of your spectrum.

The ``InfoDialog``, if open, still works to change experimental parameters and the energy scale.

In [108]:
compositionWidget = ieels.CompositionWidget(datasets)

AppLayout(children=(GridspecLayout(children=(ToggleButton(value=False, button_style='info', description='Fit A…

### Output of Results

In [109]:
edges = compositionWidget.dataset.metadata['edges']
element = []
areal_density = []
for key, edge in edges.items():
    if key.isdigit():
        element.append(edge['element'])
        areal_density.append(edge['areal_density'])

print('Relative chemical composition of ', compositionWidget.dataset.title)
for i in range(len(element)):
    print(f'{element[i]}: {areal_density[i]/np.sum(areal_density)*100:.1f} %')
    
saved_edges_metadata = edges

Relative chemical composition of  Gellerup SrTiO4 EELS Ti and O edge spectra 60mu 1s
Ti: 18.3 %
O: 81.7 %


### Log Data
All the data associated with quantification are stored in the ``metadata`` attribute. The relevant keys of that dictionary are:  ``experiment`` and  ``edges``.

In [62]:
compositionWidget.dataset.metadata.keys()

dict_keys(['experiment', 'filename', 'edges'])

In [63]:
compositionWidget.dataset.view_metadata()

experiment :
	single_exposure_time : 1.0
	exposure_time : 10.0
	number_of_frames : 10
	collection_angle : 30.0
	convergence_angle : 10.0
	microscope : 
	acceleration_voltage : 200000.0
	flux_ppm : 122228.49553999992
	count_conversion : 1
	beam_current : 0
filename : C:\Users\gduscher\Documents\Github\2023-04018-T1-3\Gellerup SrTiO4 EELS Ti and O edge spectra 60mu 1s 0.2 disp.dm3
edges :
	model :
		background : energy_loss:  energy-loss (energy_loss) of size (2048,)
		background-poly_0 : 978052.490989928
		background-poly_1 : -4334.334300390493
		background-poly_2 : 3.7359237478656087
		background-A : 55983934983.91169
		background-r : 1.8228302217049321
		spectrum : energy_loss:  energy-loss (energy_loss) of size (2048,)
		blurred : [1564512.9  1567874.5  1574252.1  ...  343700.06  343373.5   343206.78]
		mask : [0. 0. 0. ... 1. 0. 1.]
		fit_parameter : [ 1.08502547e+14  3.93631284e+14  2.98252875e+05  2.98252875e+05
  2.98252875e+05  2.98252875e+05  9.78052491e+05 -4.33433430e+03
  3.

## ELNES
The electron energy-loss near edge structure is determined by fititng the spectrum after quantification model subtraction. 

First smooth the spectrum (only 0 iteration available at this time) and then 
find the number of peaks you want (Can be repeated as oftern as one wants).

Please note that the peaks do not necessarily have a physcial meaning as there are peaks with 
positive and negative amplitudes. The goal is to get a smooth representation of the spectrum.
A difference of a good fit and the data should result in a residual of noise only. To achieve such a good fit, the careful setting of the edge onset is important.


In [101]:
peakFitWidget = ieels.PeakFitWidget(datasets)

AppLayout(children=(GridspecLayout(children=(Button(description='Fit Area', layout=Layout(grid_area='widget001…

### Output

In [104]:
areas = []
for p, peak in peakFitWidget.peaks['peaks'].items():
    area = np.sqrt(2* np.pi)* peak['amplitude'] * np.abs(peak['width'] / np.sqrt(2 *np.log(2))) 
    areas.append(area)
    if 'associated_edge' not in peak:
        peak['associated_edge']= ''
    print(f"peak  {p}: position: {peak['position']:7.1f}, area: {area:12.3f} associated edge: {peak['associated_edge']}")
#print(f'\n M4/M5 peak 2 to peak 1 ratio: {(areas[1])/areas[0]:.2f}')

peak  0: position:   466.4, area:  2038764.974 associated edge: Ti-L2
peak  1: position:   460.9, area:  1427476.665 associated edge: Ti-L3
peak  2: position:   464.1, area:  1238911.015 associated edge: Ti-L2
peak  3: position:   458.7, area:   578044.335 associated edge: Ti-L3
peak  4: position:   531.9, area:   369866.282 associated edge: 
peak  5: position:   544.5, area:    57486.603 associated edge: O-K1


### Log Data
All the data associated with quantification are stored in the ``metadata`` attribute. 
The new key in that dictionary is: ``peak_fit``.

In [55]:
infoWidget.dataset.metadata.keys()

dict_keys(['filename', 'edges', 'experiment'])

## Save and Close File
The dataset with the metadata should be saved for further quantification. 
The file needs to be closed to be used with other notebooks.

In [105]:
h5_group = ft.save_dataset(infoWidget.datasets, '../../example_data/EELS_STO3.hf5')
h5_group.file.close()

C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future version',
C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future version',
C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_io.py:111: UserWarning: main_data_name should not contain the "-" character. Reformatted name from:collection-60muem to collection_60muem
  warn('main_data_name should not contain the "-" character. Reformatted'
C:\Users\gduscher\AppData\Local\anaconda3\envs\pyter\lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future